# Model Training

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import os
import sys
module = os.path.abspath('/home/learner/DLA_project/src/main')
# module = os.path.abspath("C:/Users\\18145\\development\\wesad_experiments\\src\\main")
if module not in sys.path:
    sys.path.append(module)
from DataManager import DataManager

In [2]:
manager = DataManager()

In [ ]:

manager.load_all()


Loading data from S2
	Path=/media/learner/6663-3462/WESAD/S2/S2.pkl
Loading data from S3
	Path=/media/learner/6663-3462/WESAD/S3/S3.pkl
Loading data from S4
	Path=/media/learner/6663-3462/WESAD/S4/S4.pkl
Loading data from S5
	Path=/media/learner/6663-3462/WESAD/S5/S5.pkl
Loading data from S6
	Path=/media/learner/6663-3462/WESAD/S6/S6.pkl
Loading data from S7
	Path=/media/learner/6663-3462/WESAD/S7/S7.pkl
Loading data from S8
	Path=/media/learner/6663-3462/WESAD/S8/S8.pkl


In [ ]:
manager.compute_features();
manager.compute_features_stress();

In [ ]:
print("We have", len(manager.SUBJECTS), " subjects")

for feature in manager.FEATURES.keys():
    print("there are ", len(manager.FEATURES[feature]), " values for ", feature)

In [ ]:
# Lets go ahead and reshape this data 
X1 = (   manager.FEATURES['a_mean'], manager.FEATURES['a_std'], manager.FEATURES['a_maxx'], manager.FEATURES['a_maxy'], manager.FEATURES['a_maxz'],\
                  manager.FEATURES['e_max'], manager.FEATURES['e_min'], manager.FEATURES['e_mean'], manager.FEATURES['e_range'], manager.FEATURES['e_std'],\
                  manager.FEATURES['t_max'], manager.FEATURES['t_min'], manager.FEATURES['t_mean'], manager.FEATURES['t_range'], manager.FEATURES['t_std'] )
X1 = np.reshape(X1, (len(manager.FEATURES['a_mean'])  ,15))

X2 = (   manager.STRESS_FEATURES['a_mean'], manager.STRESS_FEATURES['a_std'], manager.STRESS_FEATURES['a_maxx'], manager.STRESS_FEATURES['a_maxy'], manager.STRESS_FEATURES['a_maxz'],\
                  manager.STRESS_FEATURES['e_max'], manager.STRESS_FEATURES['e_min'], manager.STRESS_FEATURES['e_mean'], manager.STRESS_FEATURES['e_range'], manager.STRESS_FEATURES['e_std'],\
                  manager.STRESS_FEATURES['t_max'], manager.STRESS_FEATURES['t_min'], manager.STRESS_FEATURES['t_mean'], manager.STRESS_FEATURES['t_range'], manager.STRESS_FEATURES['t_std'] )
X2 = np.reshape(X2, (len(manager.STRESS_FEATURES['a_mean'])  ,15))


In [ ]:
# import pip

# try:
#     import keras
# except ImportError:
#     ! sudo pip3 install keras
# try:
#     import h5py
# except ImportError:
#     ! sudo pip3 install h5py
# try:
#     import ibmiotf
# except ImportError:
#     ! sudo pip3 install ibmiotf
# try:
#     import tensorflow
# except ImportError:
#     ! sudo pip3 install tensorflow

In [ ]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from  sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import tensorflow
import keras
from keras.preprocessing import sequence

from keras.models import Sequential

from keras.layers import Dense, Embedding, Flatten

from keras.layers import LSTM

It will be better to have our data scaled between 0 and 1, so lets go ahead and create a function to 
normalize the data that way now.

In [ ]:
def normalize(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

In [ ]:
# x1 is the baseline data
# x2 is the stress data

y1 = [0] * len(X1)
y2 = [1] * len(X2)

In [ ]:
print(len(y1))
print(len(y2))

Now we need to concat the data between baseline and stress so that we can split it into training and test sets

In [ ]:
X = np.concatenate((X1, X2), axis=0)
print(np.shape(X))

y = np.concatenate((y1,y2), axis=0)
print(np.shape(y))

In [ ]:
# Split the data up in train and test sets
from sklearn.model_selection import train_test_split

# since we don't have the same number of samples for x and y, 
# we will drop off some x values for creating the sample
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

In [ ]:
# Should be

# input_shape=(number of sequences=?, time_steps=None, features=15)

# target=(number of sequences, time_steps, targets)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

In [ ]:
print('Building the LSTM network...')


model = Sequential()

model.add(LSTM(32, input_shape=(1, 15), return_sequences=True))

# model.add(LSTM(15, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))

# https://stackoverflow.com/questions/49623436/valueerror-error-when-checking-target-expected-dense-2-to-have-3-dimensions-b
# mode.add(Flatten())
model.add(Dense(y.shape[1], activation='sigmoid'))


In [ ]:
model.summary()

print("inputs: " , model.input_shape)

print("outputs: ", model.output_shape)

print("actual inputs: ")

print("actual outputs: ")

In [ ]:
model.compile(loss='binary_crossentropy',

              optimizer='adam',

              metrics=['accuracy'])


In [ ]:
print('Training LSTM...')

batch_size = 1 # I think I want to use batch_size = 1


model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test,

                            batch_size=batch_size)

In [ ]:
print('score:', score)

print('accuracy:', acc)

In [ ]:
# score, acc = model.evaluate(x_test, y_test,

#                             batch_size=batch_size)

print('Test score:', score)

print('Test accuracy:', acc)

In [ ]:
print((x_train))


In [ ]:
print(len(x_train[0]))
print(type(x_train))

In [ ]:
#manager.train_model()

# manager.test_model()
